In [1]:
from mlx_utils import build_model, mlx_encode

/Users/rickyhsu/miniconda3/envs/rag-label/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model, tokenizer = build_model('model_cfg.json', weight_path='mlx_bert.npz')
texts = ['今天天氣如何']
seq_embed, pooled_embed, attn_mask = mlx_encode(texts[0], tokenizer, model)

In [3]:
from transformers import AutoModel, AutoTokenizer
import torch.nn.functional as F
from torch import Tensor
def average_pool(last_hidden_states: Tensor,
    attention_mask: Tensor) -> Tensor:
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]
model_name = 'intfloat/multilingual-e5-small'
max_length = 512
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name).to('cpu')
model.eval()
batch_dict = tokenizer([texts[0]], max_length=512, padding=True, truncation=True, return_tensors='pt')
outputs = model(**batch_dict)
# print('outputs')
embeddings = average_pool(outputs.last_hidden_state, batch_dict['attention_mask'])
embeddings = F.normalize(embeddings, p=2, dim=1).tolist()

In [19]:
import torch
mlx_embed = torch.tensor(seq_embed)
mlx_embed = average_pool(mlx_embed, torch.tensor(attn_mask))
mlx_embed = F.normalize(mlx_embed, p=2, dim=1).tolist()

In [23]:
from sentence_transformers import util
tmp = [mlx_embed[0], embeddings[0]]
util.cos_sim(tmp, tmp)

tensor([[1.0000, 1.0000],
        [1.0000, 1.0000]])